In [1]:
import pickle

with open('user_pos_test.pkl', 'rb') as f:
    user_pos_test = pickle.load(f)

with open('test_items.pkl', 'rb') as f:
    test_items = pickle.load(f)

with open('rating.pkl', 'rb') as f:
    rating = pickle.load(f)

In [2]:
user_pos_test

[61608]

In [4]:
len(test_items)

208334

In [6]:
len(rating)

208361